In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Parkinsons_Disease"
cohort = "GSE101534"

# Input paths
in_trait_dir = "../../input/GEO/Parkinsons_Disease"
in_cohort_dir = "../../input/GEO/Parkinsons_Disease/GSE101534"

# Output paths
out_data_file = "../../output/preprocess/Parkinsons_Disease/GSE101534.csv"
out_gene_data_file = "../../output/preprocess/Parkinsons_Disease/gene_data/GSE101534.csv"
out_clinical_data_file = "../../output/preprocess/Parkinsons_Disease/clinical_data/GSE101534.csv"
json_path = "../../output/preprocess/Parkinsons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genome-wide expression profiling of the LRRK2-G2019S mutation in hNES cells"
!Series_summary	"Parkinson’s disease (PD) has a neuro-developmental component with multiple genetic predispositions. The most prevalent mutation, LRRK2-G2019S is linked to familial and sporadic PD. Based on the multiple origins of PD and the incomplete penetrance of LRRK2-G2019S, we hypothesize that modifiers in the patient genetic background act as susceptibility factors for developing PD. To assess the developmental component of LRRK2-G2019S pathogenesis, we used 19 human iPSC-derived neuroepithelial stem cell lines (NESCs). Isogenic controls distinguish between LRRK2-G2019S dependent and independent cellular phenotypes. LRRK2-G2019S patient and healthy mutagenized lines showed altered NESC self-renewal. Within patients, phenotypes were only partly LRRK2-G2019S dependent, suggesting Parkinson’s disease (PD) has a neuro-developmental component with multiple genetic predi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import json
from typing import Dict, Any, Optional, Callable

# Review the sample characteristics dictionary
# This dataset is about LRRK2-G2019S mutation in Parkinson's Disease

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains genome-wide expression profiling data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# From the sample characteristics dictionary, we see key 0 contains mutation status
# The values include 'healthy', 'patient', 'gene corrected', and 'inserted G2019S'
# The 'patient' vs 'healthy' can be used as our trait value (PD status)
trait_row = 0  # Key in the sample characteristics dictionary

# There is no age data available
age_row = None

# There is no gender data available
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """
    Convert trait value (PD mutation status) to binary.
    0: Healthy/gene corrected (no PD)
    1: Patient/inserted G2019S (PD or PD mutation)
    """
    if value is None:
        return None
    
    # Extract the value after colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.lower() == "healthy" or value.lower() == "gene corrected":
        return 0
    elif value.lower() == "patient" or value.lower() == "inserted g2019s":
        return 1
    else:
        return None

# Age conversion function - not used but defined for completeness
def convert_age(value: str) -> Optional[float]:
    return None

# Gender conversion function - not used but defined for completeness
def convert_gender(value: str) -> Optional[int]:
    return None

# 3. Save Metadata
# Initial filtering on the usability of the dataset
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Ensure the directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Use the function from the library to extract clinical features
    # Assuming clinical_data is already loaded from a previous step
    try:
        # Check if clinical_data is already defined
        clinical_data
    except NameError:
        # If not defined, we need to create a dataframe based on the sample characteristics
        # As a placeholder, using the provided information
        clinical_data = pd.DataFrame({0: ['mutation: healthy', 'mutation: patient', 'mutation: gene corrected', 'mutation: inserted G2019S']})
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Clinical features preview:", preview)
    
    # Save the clinical features to a CSV file
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Clinical features preview: {'GSM2705776': [0.0], 'GSM2705777': [1.0], 'GSM2705778': [1.0], 'GSM2705779': [0.0], 'GSM2705780': [0.0], 'GSM2705781': [0.0], 'GSM2705782': [1.0], 'GSM2705783': [1.0], 'GSM2705784': [1.0], 'GSM2705785': [0.0], 'GSM2705786': [0.0], 'GSM2705787': [0.0], 'GSM2705788': [0.0], 'GSM2705789': [1.0], 'GSM2705790': [1.0], 'GSM2705791': [1.0], 'GSM2705792': [0.0], 'GSM2705793': [0.0], 'GSM2705794': [0.0], 'GSM2705795': [0.0], 'GSM2705796': [0.0], 'GSM2705797': [0.0], 'GSM2705798': [0.0], 'GSM2705799': [0.0], 'GSM2705800': [0.0], 'GSM2705801': [0.0], 'GSM2705802': [1.0], 'GSM2705803': [1.0], 'GSM2705804': [1.0], 'GSM2705805': [1.0], 'GSM2705806': [1.0], 'GSM2705807': [0.0], 'GSM2705808': [0.0], 'GSM2705809': [0.0], 'GSM2705810': [1.0], 'GSM2705811': [1.0], 'GSM2705812': [0.0], 'GSM2705813': [0.0], 'GSM2705814': [1.0], 'GSM2705815': [0.0], 'GSM2705816': [1.0], 'GSM2705817': [1.0], 'GSM2705818': [1.0], 'GSM2705819': [0.0], 'GSM2705820': [0.0], 'GSM2705821': [0.0], 'GSM27

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step
# These appear to be numeric identifiers (likely probe IDs) rather than standard human gene symbols
# Human gene symbols typically have alphabetic characters (like BRCA1, TP53, etc.)
# Therefore, these identifiers need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examining the gene annotation data and gene_data indexes to determine mapping columns
print("Gene annotation columns:", gene_annotation.columns.tolist())
print("Sample gene data index:", gene_data.index[:5])

# Print some diagnostic information about our data
print(f"Number of IDs in gene_data: {len(gene_data.index)}")
print(f"Number of IDs in gene_annotation: {len(gene_annotation)}")

# Check column contents for potential gene identifiers
print("\nChecking potential gene identifier columns:")
for col in gene_annotation.columns:
    non_null_count = gene_annotation[col].count()
    sample_values = gene_annotation[col].dropna().head(3).tolist()
    print(f"Column '{col}': {non_null_count} non-null values, samples: {sample_values}")

# Create a mapping dataframe using accession numbers directly without trying to extract symbols
# We'll use GB_ACC directly as our gene identifiers since they're RefSeq/GenBank accessions
mapping_data = gene_annotation[['ID', 'GB_ACC']].dropna()
mapping_data = mapping_data.rename(columns={'GB_ACC': 'Gene'})
mapping_data['Gene'] = mapping_data['Gene'].astype(str)  # Ensure gene IDs are strings

# Print sample of mapping data to verify
print("\nGene mapping preview (using accession numbers directly):")
print(preview_df(mapping_data))
print(f"Number of probes with gene information: {len(mapping_data)}")

# 3. Apply the mapping to convert probe-level measurements to gene expression data
# Modify the function call to bypass the extract_human_gene_symbols step
# We'll create a custom function that wraps apply_gene_mapping but skips symbol extraction
def apply_direct_gene_mapping(expression_df, mapping_df):
    """Apply gene mapping without attempting to extract gene symbols."""
    mapping_df = mapping_df[mapping_df['ID'].isin(expression_df.index)].copy()
    mapping_df['num_genes'] = 1  # Each accession is treated as one gene
    mapping_df.set_index('ID', inplace=True)
    
    # Merge and distribute expression values
    merged_df = mapping_df.join(expression_df)
    expr_cols = [col for col in merged_df.columns if col not in ['Gene', 'num_genes']]
    
    # Group by accession number and sum the expression values
    gene_expression_df = merged_df.groupby('Gene')[expr_cols].sum()
    
    return gene_expression_df

# Apply our direct mapping function
mapped_gene_data = apply_direct_gene_mapping(gene_data, mapping_data)

# Verify the results
print("\nGene expression data after mapping:")
print(f"Shape: {mapped_gene_data.shape}")
print(f"Number of genes: {len(mapped_gene_data.index)}")
print("Sample of genes (up to 5):", list(mapped_gene_data.index)[:5] if len(mapped_gene_data.index) > 0 else "None")

# Use the mapping result if successful, otherwise use probe IDs
if len(mapped_gene_data.index) > 0:
    gene_data = mapped_gene_data
    print("Successfully mapped probes to gene accession numbers.")
else:
    print("WARNING: Gene mapping failed. Using probe IDs directly as fallback.")
    # gene_data is already in the right format with probe IDs

# Save the processed gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene annotation columns: ['ID', 'RANGE_STRAND', 'RANGE_START', 'RANGE_END', 'total_probes', 'GB_ACC', 'SPOT_ID', 'RANGE_GB']
Sample gene data index: Index(['16650001', '16650003', '16650005', '16650007', '16650009'], dtype='object', name='ID')
Number of IDs in gene_data: 53617
Number of IDs in gene_annotation: 2788499

Checking potential gene identifier columns:
Column 'ID': 2788499 non-null values, samples: ['16657436', '16657440', '16657445']


Column 'RANGE_STRAND': 2786290 non-null values, samples: ['+', '+', '+']
Column 'RANGE_START': 51772 non-null values, samples: [12190.0, 29554.0, 69091.0]
Column 'RANGE_END': 51772 non-null values, samples: [13639.0, 31109.0, 70008.0]
Column 'total_probes': 51772 non-null values, samples: [25.0, 28.0, 8.0]
Column 'GB_ACC': 17623 non-null values, samples: ['NR_046018', 'NR_024368', 'NR_029406']
Column 'SPOT_ID': 53981 non-null values, samples: ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008']


Column 'RANGE_GB': 47833 non-null values, samples: ['NC_000001.10', 'NC_000001.10', 'NC_000001.10']

Gene mapping preview (using accession numbers directly):
{'ID': ['16657436', '16657450', '16657476', '16657485', '16657514'], 'Gene': ['NR_046018', 'NR_024368', 'NR_029406', 'XR_132471', 'NR_047526']}
Number of probes with gene information: 17623

Gene expression data after mapping:
Shape: (16346, 51)
Number of genes: 16346
Sample of genes (up to 5): ['NM_000014', 'NM_000015', 'NM_000019', 'NM_000020', 'NM_000021']
Successfully mapped probes to gene accession numbers.


Gene expression data saved to ../../output/preprocess/Parkinsons_Disease/gene_data/GSE101534.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols from the already mapped gene expression data from Step 6
# Since the gene data contains RefSeq accessions (NM_, NR_) rather than standard gene symbols,
# we'll use these directly without normalization

# Let's use the original gene_data from Step 6 instead of trying to normalize
print(f"Original gene expression data shape: {gene_data.shape}")
print("First 5 gene identifiers:")
print(gene_data.index[:5])

# Save the gene data without normalization
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was already processed in Step 2
if os.path.exists(out_clinical_data_file):
    clinical_data_processed = pd.read_csv(out_clinical_data_file)
    print("Loaded clinical data from file")
else:
    # If for some reason the file wasn't saved, recreate the clinical features using the same parameters
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Use the same conversion function and trait_row from Step 2
    def convert_trait(value):
        if value is None:
            return None
        
        # Extract the value after colon if it exists
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        # Convert to binary
        if value.lower() == "healthy" or value.lower() == "gene corrected":
            return 0
        elif value.lower() == "patient" or value.lower() == "inserted g2019s":
            return 1
        else:
            return None
    
    # Use the exact same parameters as determined in Step 2
    clinical_data_processed = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,  # From Step 2
        convert_trait=convert_trait,
        age_row=None,  # From Step 2
        convert_age=None,
        gender_row=None,  # From Step 2
        convert_gender=None
    )
    
    # Save it again just to be sure
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_data_processed.to_csv(out_clinical_data_file, index=True)

# Ensure clinical data has the right format
# The expected format should have samples as rows and features as columns
# Adjust if needed to match the expected format for the linking step
if clinical_data_processed.shape[0] == 1:
    # If it's structured with samples as columns and 1 row for the trait
    # Transpose to get samples as rows and features as columns
    clinical_data_processed = clinical_data_processed.T
    clinical_data_processed.columns = [trait]  # Rename column to match the trait
    print("Transposed clinical data to have samples as rows")

print("Clinical data shape:", clinical_data_processed.shape)
print("Clinical data preview:")
print(preview_df(clinical_data_processed))

# 3. Link clinical and genetic data
# We need to transpose gene_data to match samples as rows
gene_data_t = gene_data.T
print(f"Transposed gene data shape: {gene_data_t.shape}")

# Now combine clinical and genetic data horizontally (samples as rows, features as columns)
if not clinical_data_processed.index.equals(gene_data_t.index):
    print("WARNING: Sample IDs don't match between clinical and gene data!")
    # Get common samples
    common_samples = clinical_data_processed.index.intersection(gene_data_t.index)
    print(f"Found {len(common_samples)} common samples")
    if len(common_samples) > 0:
        clinical_data_processed = clinical_data_processed.loc[common_samples]
        gene_data_t = gene_data_t.loc[common_samples]

# Combine data horizontally
linked_data = pd.concat([clinical_data_processed, gene_data_t], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
# Use handle_missing_values with the correct trait column name
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")
print(f"Is the trait biased: {is_biased}")

# 6. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data related to LRRK2-G2019S mutation in Parkinson's Disease."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable. Linked data was not saved.")

Original gene expression data shape: (16346, 51)
First 5 gene identifiers:
Index(['NM_000014', 'NM_000015', 'NM_000019', 'NM_000020', 'NM_000021'], dtype='object', name='Gene')


Gene data saved to ../../output/preprocess/Parkinsons_Disease/gene_data/GSE101534.csv
Loaded clinical data from file
Transposed clinical data to have samples as rows
Clinical data shape: (51, 1)
Clinical data preview:
{'Parkinsons_Disease': [0.0, 1.0, 1.0, 0.0, 0.0]}
Transposed gene data shape: (51, 16346)
Linked data shape: (51, 16347)


Data shape after handling missing values: (51, 16347)
For the feature 'Parkinsons_Disease', the least common label is '1.0' with 24 occurrences. This represents 47.06% of the dataset.
The distribution of the feature 'Parkinsons_Disease' in this dataset is fine.

Data shape after removing biased features: (51, 16347)
Is the trait biased: False
A new JSON file was created at: ../../output/preprocess/Parkinsons_Disease/cohort_info.json


Linked data saved to ../../output/preprocess/Parkinsons_Disease/GSE101534.csv
